# Exercise 2

Below you find the whole code of the MFA System we just discussed. 

Make the following modifications: 

## Exercises

### 2A: Add a `waste` stock.
- Adapt the definitions 
  - Introduce the stock definition:
    - The `subclass` should be `fd.SimpleFlowDrivenStock`. It needs no lifetime model.
  - Introduce an according process
  - Change the destination of the `use => sysenv` flow from `sysenv` to this process in the `FlowDefinition`.
- Adapt the compute routine
  - change the name of the `use => sysenv`, following its new destination
  - set this flow as the `inflow` of the new waste stock.
  - call the `compute` routine of the stock. 
    - With an input given, the `compute()` function of this stock just accumulates the inflows over time.

### 2B: Add an `element` dimension.
- Add the dimension definition (Name must be `element`, letter could be `e`)
- Add the parameter definition `element shares` (with `element` being its only dimension)
- Add the file locations of the pre-existing files `data/dimension_element.csv` and `data/parameter_element_shares.csv` to the according dictionaries
- Add the dimension letter to all flows and stocks.
- In the first line of the `compute` function, multiply the right-hand side (parameter `production`) with the new parameter `element shares`.

### 2C: Export a stock and/or a flow
- Either by plotting it
- Or by exporting to a dataframe and printing to stdout. 

## Code

### Definitions

In [ ]:
import flodym as fd

dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int),
    fd.DimensionDefinition(letter="p", name="product", dtype=str),
]

In [ ]:
parameter_definitions = [
    fd.ParameterDefinition(name="production", dim_letters=("t",)),
    fd.ParameterDefinition(name="manufacturing loss rate", dim_letters=("t",)),
    fd.ParameterDefinition(name="product shares", dim_letters=("p",)),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("p",)),
]

In [ ]:
process_names = [
    "sysenv",
    "manufacturing",
    "use",
    "waste",
]

In [ ]:
flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="manufacturing", dim_letters=("t",)),
    fd.FlowDefinition(from_process_name="manufacturing", to_process_name="sysenv", dim_letters=("t",)),
    fd.FlowDefinition(from_process_name="manufacturing", to_process_name="use", dim_letters=("t", "p")),
    fd.FlowDefinition(from_process_name="use", to_process_name="waste", dim_letters=("t", "p")),
]

In [ ]:
stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("t", "p"),
        subclass=fd.InflowDrivenDSM,
        lifetime_model_class=fd.LogNormalLifetime,
    ),
        fd.StockDefinition(
        name="waste",
        process="waste",
        dim_letters=("t",),
        subclass=fd.SimpleFlowDrivenStock,
    ),
]

In [ ]:
mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

### Data sources 

In [ ]:
dimension_files = {
    "time": "data/dimension_time.csv",
    "product": "data/dimension_product.csv",
}

In [ ]:
parameter_files = {
    "production": "data/parameter_production.csv",
    "manufacturing loss rate": "data/parameter_manufacturing_loss_rate.csv",
    "product shares": "data/parameter_product_shares.csv",
    "product lifetimes": "data/parameter_product_lifetimes.csv",
}


### `compute` routine

In [ ]:
class SimpleMFA(fd.MFASystem):
    def compute(self):

        # manufacturing flows
        self.flows["sysenv => manufacturing"][...] = self.parameters["production"]
        self.flows["manufacturing => sysenv"][...] = self.flows["sysenv => manufacturing"] * self.parameters["manufacturing loss rate"]
        total_products = self.flows["sysenv => manufacturing"] - self.flows["manufacturing => sysenv"]
        self.flows["manufacturing => use"][...] = total_products * self.parameters["product shares"]

        # use stock
        self.stocks["use"].inflow[...] = self.flows["manufacturing => use"]
        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.5*self.parameters["product lifetimes"],
        )
        self.stocks["use"].compute()

        # end-of-life  flow
        self.flows["use => waste"][...] = self.stocks["use"].outflow

        # waste stock
        self.stocks["waste"].inflow[...] = self.flows["use => waste"][...]
        self.stocks["waste"].compute()


### Init, load & compute

In [ ]:
mfa_example = SimpleMFA.from_csv(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
)

In [ ]:
mfa_example.compute()

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

mfa_example.check_mass_balance()

### Sankey Plotting

In [ ]:
import flodym.export as fde

fig = fde.PlotlySankeyPlotter(mfa=mfa_example, exclude_processes=[]).plot()
fig.show()